# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter



# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
print("Total NANs in the dataframe: ", df.isna().sum().sum())

df.head()

Total NANs in the dataframe:  0


,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# Rename columns 
df.rename(
    columns = 
    {"homeowner" : "hm",
    "loan_status": "l_s"}, 
          inplace =True)
df.head()

,loan_size,interest_rate,hm,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,l_s
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
# Check categories of the homeowners
df['hm'].value_counts()

mortgage    38572
own         30930
rent         8034
Name: hm, dtype: int64

In [6]:
# Check categories of the loan status
df['l_s'].value_counts()

low_risk     75036
high_risk     2500
Name: l_s, dtype: int64

In [7]:
# Home owner dictionary
homeowner = {
    "mortgage": 0,
    "own" : 1,
    "rent": 2
}

# Loan status dictionary
loan_status = {
    "high_risk": 0, # Likely bad loans
    "low_risk" : 1 # Likely good loans
}


# Homeowner encoded using the dictionary values
df["homeowner"] = df["hm"].apply(lambda x: homeowner[x])

# loan status encoded using the dictionary values
df["loan_status"] = df["l_s"].apply(lambda x: loan_status[x])

# drop duplicate columns 
df.drop(["hm", "l_s"], axis = 1, inplace = True)

# Check the dataframe form
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,1,1
1,8400.0,6.692,43600,0.311927,3,0,13600,1,1
2,9000.0,6.963,46100,0.349241,3,0,16100,2,1
3,10700.0,7.664,52700,0.430740,5,1,22700,1,1
4,10800.0,7.698,53000,0.433962,5,1,23000,0,1


# Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop(columns = "loan_status") 

# Create our target as a dataframe
y = pd.DataFrame(df["loan_status"], columns = ["loan_status"])

# Check the form of y
display(y.head())


,loan_status
0,1
1,1
2,1
3,1
4,1


In [9]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.606144
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.667811
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,1.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,2.000000


In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [11]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

# since y is a dataframe data needs to be extrated as an array
# (y.values) and sometimes you may need to reshape the array 
# (y.values.reshape (-1, 1))
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                   y.values, 
                                                   random_state=1, 
                                                   stratify=y)

In [12]:
X_train.shape

(58152, 8)

In [13]:
X_test.shape

(19384, 8)

In [14]:
y_train.shape

(58152, 1)

In [15]:
y_test.shape

(19384, 1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Importing the StandardScaler instance
from sklearn.preprocessing import StandardScaler

# Create the StandardScaler instance  to account for outliers 
# and to process data to feed into predictive models
scaler = StandardScaler()

In [17]:
# Fit the Standard Scaler with the training data
# when fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [18]:
# Scale the training and testing data 
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# The transformer above gives out an array that needs to be converted back to the 
# dataframe format used before this transform
X_train=pd.DataFrame(X_train, columns = 
                     ["loan_size",
                      "interest_rate",
                      "borrower_income",
                      "debt_to_income",
                      "num_of_accounts",
                      "derogatory_marks",
                      "total_debt",
                      "homeowner"])
X_test=pd.DataFrame(X_test, columns =
                    ["loan_size",
                     "interest_rate",
                     "borrower_income",
                     "debt_to_income",
                     "num_of_accounts",
                     "derogatory_marks",
                     "total_debt",
                     "homeowner"])

In [19]:
X_train.shape

(58152, 8)

In [20]:
X_test.shape

(19384, 8)

# Simple Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)

# Train the data
model.fit(X_train, y_train)

# Predict outcomes for test data set
y_pred = model.predict(X_test)

In [22]:
# Score the model
print("Training Data Score: {0:.6f}".format(model.score(X_train, y_train)))
print("Testing Data Score: {0:.6f}".format(model.score(X_test, y_test)))

Training Data Score: 0.994102
Testing Data Score: 0.994222


In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

# Balanced accuracy score
balanced_accuracy_score = balanced_accuracy_score(y_test, y_pred)

# Balanced accuracy score
print("Simple Logistic Regression Balanced Accuracy Score: {0:.8f}".format(balanced_accuracy_score))

Simple Logistic Regression Balanced Accuracy Score: 0.98928130


In [24]:
# Creating the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_array = confusion_matrix(y_test, y_pred)

print("Confusion Matrix: \n")

# Converting the Confusion Matrix to a dataframe
confusion_df = pd.DataFrame(
    confusion_array, 
    index = ["High Risk [0] ", "Low Risk [1]"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying the Confusion Matrix
display(confusion_df.head())

Confusion Matrix: 



,Predicted 0,Predicted 1
High Risk [0],615,10
Low Risk [1],102,18657


In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print("Simple Logistic Regression Imbalanced Classification Report: \n")
target_names = ["High Risk [0]", "Low Risk [1]"]
print(classification_report_imbalanced(y_test, y_pred, target_names=target_names))

Simple Logistic Regression Imbalanced Classification Report: 

                     pre       rec       spe        f1       geo       iba       sup

High Risk [0]       0.86      0.98      0.99      0.92      0.99      0.98       625
 Low Risk [1]       1.00      0.99      0.98      1.00      0.99      0.98     18759

  avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [26]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
ROS_X_resampled, ROS_y_resampled = ros.fit_resample(X_train, y_train) # ROS = Random Over Sampler

# View the count of target classes with Counter
Counter(ROS_y_resampled)

Counter({1: 56277, 0: 56277})

In [27]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(ROS_X_resampled, ROS_y_resampled)

# Generating model predictions
ROS_y_pred = model.predict(X_test)

In [28]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

ROS_balanced_accuracy_score = balanced_accuracy_score(y_test, ROS_y_pred)

print("Random Over Sampling Balanced Accuracy Score: {0:.8f}".format(ROS_balanced_accuracy_score))

Random Over Sampling Balanced Accuracy Score: 0.99464142


In [29]:
# Generating and displaying the confusion matrix
from sklearn.metrics import confusion_matrix

ROS_confusion_array = confusion_matrix(y_test, ROS_y_pred)

print("Random Over Sampling Confusion Matrix: \n")

# Converting the Confusion Matrix to a dataframe
ROS_confusion_df = pd.DataFrame(
    ROS_confusion_array, 
    index = ["High Risk [0] ", "Low Risk [1]"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying the Confusion Matrix
display(ROS_confusion_df.head())

Random Over Sampling Confusion Matrix: 



,Predicted 0,Predicted 1
High Risk [0],622,3
Low Risk [1],111,18648


In [30]:
# Print the imbalanced classification report
print("Random Over Sampling Imbalanced Classification Report: \n")
target_names = ["High Risk [0]", "Low Risk [1]"]
print(classification_report_imbalanced(y_test, ROS_y_pred, target_names = target_names))

Random Over Sampling Imbalanced Classification Report: 

                     pre       rec       spe        f1       geo       iba       sup

High Risk [0]       0.85      1.00      0.99      0.92      0.99      0.99       625
 Low Risk [1]       1.00      0.99      1.00      1.00      0.99      0.99     18759

  avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [31]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

SMOTE_X_resampled, SMOTE_y_resampled = SMOTE(random_state=1, 
                sampling_strategy=1.0).fit_resample(
                X_train, y_train
)

# View the count of target classes with Counter
Counter(SMOTE_y_resampled)

Counter({1: 56277, 0: 56277})

In [32]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(SMOTE_X_resampled, SMOTE_y_resampled)
# Predictions from the model
SMOTE_y_pred = model.predict(X_test)

In [33]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

SMOTE_balanced_accuracy_score = balanced_accuracy_score(y_test, SMOTE_y_pred)

print("SMOTE Balanced Accuracy Score: {0:.8f}".format(SMOTE_balanced_accuracy_score))

SMOTE Balanced Accuracy Score: 0.99464142


In [34]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

SMOTE_confusion_array = confusion_matrix(y_test, SMOTE_y_pred)

print("Synthetic Minority Over Sampling Confusion Matrix: \n")

# Converting the Confusion Matrix to a dataframe
SMOTE_confusion_df = pd.DataFrame(
    SMOTE_confusion_array, 
    index = ["High Risk [0] ", "Low Risk [1]"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying the Confusion Matrix
display(SMOTE_confusion_df.head())

Synthetic Minority Over Sampling Confusion Matrix: 



,Predicted 0,Predicted 1
High Risk [0],622,3
Low Risk [1],111,18648


In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print("SMOTE Imbalanced Classification Report: \n")

target_names = ["High Risk [0]", "Low Risk [1]"]
print(classification_report_imbalanced(y_test, SMOTE_y_pred, target_names = target_names))

SMOTE Imbalanced Classification Report: 

                     pre       rec       spe        f1       geo       iba       sup

High Risk [0]       0.85      1.00      0.99      0.92      0.99      0.99       625
 Low Risk [1]       1.00      0.99      1.00      1.00      0.99      0.99     18759

  avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
CC_X_resampled, CC_y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(CC_y_resampled)

Counter({0: 1875, 1: 1875})

In [37]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(CC_X_resampled, CC_y_resampled)
# Predictions from the model
CC_y_pred = model.predict(X_test)

In [38]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

CC_balanced_accuracy_score = balanced_accuracy_score(y_test, CC_y_pred)

print("ClusterCentroid Under Sampling Balanced Accuracy Score: {0:.8f}".format( 
      CC_balanced_accuracy_score))

ClusterCentroid Under Sampling Balanced Accuracy Score: 0.99328130


In [39]:
# Display the confusion matrix
CC_confusion_array = confusion_matrix(y_test, CC_y_pred)

print("ClusterCentroid Under Sampling Confusion Matrix: \n")

# Converting the Confusion Matrix to a dataframe
CC_confusion_df = pd.DataFrame(
    CC_confusion_array, 
    index = ["High Risk [0]", "Low Risk [1]"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying the Confusion Matrix
display(CC_confusion_df.head())

ClusterCentroid Under Sampling Confusion Matrix: 



,Predicted 0,Predicted 1
High Risk [0],620,5
Low Risk [1],102,18657


In [40]:
# Print the imbalanced classification report
print("ClusterCentroid Under Sampling Imbalanced Classification Report: \n")
target_names = ["High Risk [0]", "Low Risk [1]"]
CC_classification_report_imbalanced = classification_report_imbalanced(y_test, CC_y_pred, target_names = target_names)
print(CC_classification_report_imbalanced)

ClusterCentroid Under Sampling Imbalanced Classification Report: 

                     pre       rec       spe        f1       geo       iba       sup

High Risk [0]       0.86      0.99      0.99      0.92      0.99      0.99       625
 Low Risk [1]       1.00      0.99      0.99      1.00      0.99      0.99     18759

  avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [41]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
SM_X_resampled, SM_y_resampled = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(SM_y_resampled)

Counter({0: 55623, 1: 55917})

In [42]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(SM_X_resampled, SM_y_resampled)
# Predictions from the model
SM_y_pred = model.predict(X_test)

In [43]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

SM_balanced_accuracy_score = balanced_accuracy_score(y_test, SM_y_pred)

print("SMOTEENN Balanced Accuracy Score: {0:.8f}".format( 
      SM_balanced_accuracy_score))

SMOTEENN Balanced Accuracy Score: 0.99464142


In [44]:
# Display the confusion matrix
SM_confusion_array = confusion_matrix(y_test, SM_y_pred)

print("SMOTEENN Confusion Matrix: \n")

# Converting the Confusion Matrix to a dataframe
SM_confusion_df = pd.DataFrame(
    SM_confusion_array, 
    index = ["High Risk [0]", "Low Risk [1]"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying the Confusion Matrix
display(SM_confusion_df.head())

SMOTEENN Confusion Matrix: 



,Predicted 0,Predicted 1
High Risk [0],622,3
Low Risk [1],111,18648


In [45]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print("SMOTEENN Imbalanced Classification Report: \n")
target_names = ["High Risk [0]", "Low Risk [1]"]

SM_classification_report_imbalanced = classification_report_imbalanced(y_test, SM_y_pred, target_names = target_names)
print(SM_classification_report_imbalanced)

SMOTEENN Imbalanced Classification Report: 

                     pre       rec       spe        f1       geo       iba       sup

High Risk [0]       0.85      1.00      0.99      0.92      0.99      0.99       625
 Low Risk [1]       1.00      0.99      1.00      1.00      0.99      0.99     18759

  avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   SMOTEENN, SMOTE, Random Over Sampling gives best balanced accuracy scores while the Simple Logistic Reg. gives the worst balanced accuracy score. 

2. Which model had the best recall score?

    Average recall score is equivalent to the second decimal place for all of the models and it is 0.99. If it is only for the High Risk (bad loans) peer lending loans then the best scores are from SMOTEENN (1.00), SMOTE (1.00), and Random Over Sampling (1.00).   

3. Which model had the best geometric mean score?

    Average Geo Mean score for all the models are equalent to the second decimal point and is at 0.99. In this regard all these models behaved the same.
